In [1]:
import os
import boto3

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import sagemaker
from sagemaker import get_execution_role
import sagemaker_pyspark

role = get_execution_role()

# Configure Spark to use the SageMaker Spark dependency jars
jars = sagemaker_pyspark.classpath_jars()

classpath = ":".join(sagemaker_pyspark.classpath_jars())

# See the SageMaker Spark Github to learn how to connect to EMR from a notebook instance
spark = (
    SparkSession.builder.config("spark.driver.extraClassPath", classpath)
    .master("local[*]")
    .getOrCreate()
)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/22 15:56:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
from pyspark.sql.functions import col
from pyspark.sql.functions import udf, from_unixtime, year, weekofyear, row_number
from pyspark.sql.functions import hour, minute, countDistinct, col, sum, count, to_date, concat, lit, when,round
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType

In [4]:
side = spark.read.option("header",True).csv("s3a://tmp-coin-lookup-data/token_recommender_v14.csv")

23/11/22 15:56:31 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
side_select = side.select(['merge','combined_token_name','combined_token_symbol','token_category','token_rank','risky_flags'])

In [8]:
side_select.head()

Row(merge='0xa50709b10c328772ea24a44b13c7e23c75561ecb', combined_token_name='28', combined_token_symbol='28', token_category='Payments', token_rank='Unranked', risky_flags='0')

In [9]:
filter_ = side_select.filter(side_select.token_category != 'null')
filter_ = filter_.filter(filter_.combined_token_name != 'null')


In [10]:
filter_.select(['merge']).distinct().count()

5016

In [11]:
filled_df = filter_.fillna({'combined_token_symbol': 'nosymbol', 'token_rank': 'notranked','risky_flags':'noflags'})


In [12]:
filled_df = filled_df.na.drop()


In [13]:
filled_df.select(['merge']).distinct().count()

5016

In [15]:
selected_token = filled_df.select(['merge']).distinct().collect()

In [16]:
selected_token = [x[0] for x in selected_token]

In [17]:
result_df_not = side_select.filter(~side_select.merge.isin(selected_token))


In [18]:
result_df_not.filter(result_df_not.combined_token_name=='WETH').show()

+--------------------+-------------------+---------------------+--------------+----------+-----------+
|               merge|combined_token_name|combined_token_symbol|token_category|token_rank|risky_flags|
+--------------------+-------------------+---------------------+--------------+----------+-----------+
|0xd431b88e19c5dc3...|               WETH|                 WETH|          null|      null|       null|
|0x8584a4ee3257c99...|               WETH|                 WETH|          null|      null|       null|
|0x65fe2e2a6955d17...|               WETH|                 WETH|          null|      null|       null|
|0x431929bb437c98f...|               WETH|                 WETH|          null|      null|       null|
|0x526f0603a845e4c...|               WETH|                 WETH|          null|      null|       null|
|0x4f12b604cad9f61...|               WETH|                 WETH|          null|      null|       null|
+--------------------+-------------------+---------------------+---------

In [33]:
filter_.repartition(1).write.mode("overwrite").parquet("s3a://zarklab-token-recommender-ohio/token_lookup")


In [20]:
month1 = '202201'
month2 = '202202'
month3 = '202203'

data_raw1 = spark.read.parquet(f's3a://zarklab-hive-test/buy-sell-updated/pt_d={month1}*/*')
data_raw2 = spark.read.parquet(f's3a://zarklab-hive-test/buy-sell-updated/pt_d={month2}*/*')
data_raw3 = spark.read.parquet(f's3a://zarklab-hive-test/buy-sell-updated/pt_d={month3}*/*')

data_raw = data_raw1.union(data_raw2).union(data_raw3)

In [22]:
data_raw = data_raw.select(['timestamp','recipient','txFee_eth','buyAsset','buyQty1','buyPrice'])
data_raw.show(10)


+----------+--------------------+--------------------+--------------------+------------------+------------------+
| timestamp|           recipient|           txFee_eth|            buyAsset|           buyQty1|          buyPrice|
+----------+--------------------+--------------------+--------------------+------------------+------------------+
|1641358889|0x0b9f404c1562438...| 0.01718327807597152|0xa808b22ffd2c472...|           12000.0|1246.7953190267892|
|1641409031|0x84ea629dc0bde08...| 0.02806140009265224|0x000000000000000...|23.663252175570342|  88789.7930046148|
|1641341387|0x450cfa4da9a67d4...| 0.02569437606145329|0x2e9d63788249371...|             100.0| 4959.662899187852|
|1641397163|0x42ad5aaaf1b94ef...| 0.04911369215895385|0xc02aaa39b223fe8...|0.2144262223721564| 817.9507901818877|
|1641341588|0x78e7925d5c4e87b...| 0.03326443667645412|0x000000000000000...|  3.56073606244043| 13472.15544882411|
|1641414757|0xc9eca04d6b98da9...|0.038454483819111235|0x000000000000000...|0.99721000235

In [23]:
data_raw = data_raw.withColumn("timestamp", from_unixtime(col("timestamp")))


In [27]:
data_raw.select(['recipient']).head(5)

[Row(recipient='0x0b9f404c15624380e1683dd6d7b5acae7bbd65d6'),
 Row(recipient='0x84ea629dc0bde080dbf7a7e335d527d03279c60d'),
 Row(recipient='0x450cfa4da9a67d4d5042f296a5bbabbcf39a890c'),
 Row(recipient='0x42ad5aaaf1b94eff0776f3f7f86234dd1c124456'),
 Row(recipient='0x78e7925d5c4e87bbf29c371876ed0adcb78b4d04')]

In [19]:
# result_df = data_raw.filter(~data_raw.buyAsset.isin(selected_token))


In [28]:
# from pyspark.sql.functions import col, count, sum
# result_df_group = result_df.groupBy("buyAsset").agg(count("timestamp").alias("count"), sum("buyPrice").alias("total_value")).orderBy(col("count").desc())
# result_df_group.show(4)

In [29]:
# result_df_join = result_df_group.join(side_select, result_df_group["buyAsset"] == side_select["merge"], "left")
#result_df_join.repartition(10).write.mode("overwrite").parquet(f"s3a://zarklab-token-recommender/token_dropped/")


In [30]:
# result_df_join.repartition(10).write.mode("overwrite").parquet(f"s3a://zarklab-token-recommender/token_dropped/")


In [32]:
from pyspark.sql.functions import lower

result_df = data_raw.join(filled_df, lower(data_raw["buyAsset"]) == filled_df["merge"], "inner")
result_df.show(5)

+-------------------+--------------------+--------------------+--------------------+------------------+-----------------+--------------------+-------------------+---------------------+-------------------+----------+-----------+
|          timestamp|           recipient|           txFee_eth|            buyAsset|           buyQty1|         buyPrice|               merge|combined_token_name|combined_token_symbol|     token_category|token_rank|risky_flags|
+-------------------+--------------------+--------------------+--------------------+------------------+-----------------+--------------------+-------------------+---------------------+-------------------+----------+-----------+
|2022-01-05 18:57:11|0x84ea629dc0bde08...| 0.02806140009265224|0x000000000000000...|23.663252175570342| 88789.7930046148|0x000000000000000...|           Ethereum|                  ETH|Blockchain Networks|        #1|          0|
|2022-01-05 00:09:47|0x450cfa4da9a67d4...| 0.02569437606145329|0x2e9d63788249371...|    

In [35]:
result_df.count()

3208582

In [37]:
data_dropped = result_df.na.drop()

In [38]:
result_df.count()

3208582

In [50]:
data_dropped = data_dropped.withColumn("buyQty1",round(data_dropped.buyQty1.cast(DoubleType()),4))
data_dropped = data_dropped.withColumn("buyPrice",round(data_dropped.buyPrice.cast(DoubleType()),4))


In [51]:
data_dropped.repartition(10).write.mode("overwrite").parquet(f"s3a://zarklab-token-recommender/buy-sell-updated-{month1}-{month3}/")


In [33]:
data_dropped.count()

NameError: name 'data_dropped' is not defined

In [34]:
data_dropped.select("recipient").distinct().count()

NameError: name 'data_dropped' is not defined

In [ ]:
data_dropped.select("buyAsset").distinct().count()

# filter out unseen tokens

In [26]:
# train_data  = spark.read.parquet(f"s3a://zarklab-hive-test/buy-sell-updated-202201/")


In [27]:
# train_tokens = train_data.select("buyAsset").distinct()

In [28]:
# train_tokens.count()

4688

In [29]:
# eva_data  = spark.read.parquet(f"s3a://zarklab-hive-test/buy-sell-updated-202202/")


In [30]:
# eval_tokens = eva_data.select("buyAsset").distinct()

In [36]:
# eva_data_filter = eva_data.join(train_tokens, on="buyAsset", how="inner")


In [37]:
# eva_data_filter.select("buyAsset").distinct().count()

2787

In [38]:
# eva_data_filter.write.mode("overwrite").parquet(f"s3a://zarklab-hive-test/buy-sell-updated-202202_filter/")


In [28]:
import pandas as pd

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


/tmp/ipykernel_8645/2094962876.py:1: DtypeWarning: Columns (1,2,3,4,5,7,9,10,11,12,13,14,15,17,18,19,20,21,22,28,33,38,43,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  infor = pd.read_csv('s3://zarklab-hive-test/ads_base_extdat_tokenrec_dm/tonen_info_ds_u11.csv')


In [31]:
infor.head()

,merge,token_website,token_logo_small,token_logo_thumb,token_logo_large,token_token_color_hex_1,token_token_color_percent_1,token_token_color_hex_2,token_token_color_percent_2,token_platform,...,whitelist_flag,honeypot_flag_red,trading_cd_flag,mint_flag,owner_balance_flag_red,hidden_owner_flag,external_call_flag,anti_whale_limit_txn_flag,self_destruct_flag_red,anti_whale_modify_flag
0,0xa50709b10c328772ea24a44b13c7e23c75561ecb,"['https://twentyeight.io/', 'https://linktr.ee...",https://assets.coingecko.com/coins/images/3214...,https://assets.coingecko.com/coins/images/3214...,https://assets.coingecko.com/coins/images/3214...,#943C23,1.583993,#C54B22,1.250521,ethereum,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0xe4efdd2eb216a4620cfa55c5cc67bd09dc64ff24,['https://www.bitcoinupside.cash/'],https://assets.coingecko.com/coins/images/3153...,https://assets.coingecko.com/coins/images/3153...,https://assets.coingecko.com/coins/images/3153...,#FFCB44,43.869892,#F2D27D,7.506255,ethereum,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0x45245bc59219eeaaf6cd3f382e078a461ff9de7b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0x3026d446ed0bdbcedac01996b7960d04a7a02528,['https://thex.cash/'],https://s2.coinmarketcap.com/static/img/coins/...,https://s2.coinmarketcap.com/static/img/coins/...,https://s2.coinmarketcap.com/static/img/coins/...,#2B1413,6.395851,#252833,2.823394,Ethereum,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0x68a118ef45063051eac49c7e647ce5ace48a68a5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
eth_df.head()

,merge,token_website,token_logo_small,token_logo_thumb,token_logo_large,token_token_color_hex_1,token_token_color_percent_1,token_token_color_hex_2,token_token_color_percent_2,token_platform,...,whitelist_flag,honeypot_flag_red,trading_cd_flag,mint_flag,owner_balance_flag_red,hidden_owner_flag,external_call_flag,anti_whale_limit_txn_flag,self_destruct_flag_red,anti_whale_modify_flag
1758,0x0000000000000000000000000000000000000eth,"['https://www.ethereum.org/', 'https://en.wiki...",https://s2.coinmarketcap.com/static/img/coins/...,https://s2.coinmarketcap.com/static/img/coins/...,https://s2.coinmarketcap.com/static/img/coins/...,#A164FF,0.144259,#7FFFFF,0.057703,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
eth_df[['merge','combined_token_name','combined_token_symbol','token_category','token_rank','risky_flags']].values

array([['0x0000000000000000000000000000000000000eth', 'Ethereum', 'ETH',
        'Blockchain Networks', '#1', 0.0]], dtype=object)